In [ ]:
import random
import matplotlib.pyplot as plt

# 初始化
num_people = 100
initial_money = 100
rounds = 10000
money = [initial_money] * num_people

def simulate_giving_money(money, rounds):
    for round in range(rounds):
        for i in range(num_people):
            if money[i] > 0:  # 只有有钱的人才能给钱
                money[i] -= 1
                recipient = random.randint(0, num_people - 1)
                money[recipient] += 1
        if round == 4999 or round == 9999:
            # 统计第5000回合和第10000回合的资金分布
            plot_distribution(money, round + 1)

def plot_distribution(money, round):
    plt.hist(money, bins=range(0, max(money) + 1), edgecolor='black')
    plt.title(f'Money Distribution after {round} Rounds')
    plt.xlabel('Amount of Money')
    plt.ylabel('Number of People')
    plt.show()

simulate_giving_money(money, rounds)

In [ ]:
import threading
import time
import random
from datetime import datetime

class LoggerThread(threading.Thread):
    def __init__(self, id, num_prints):
        super().__init__()
        self.id = id
        self.num_prints = num_prints
        self.stop_event = threading.Event()

    def run(self):
        while not self.stop_event.is_set():
            self.print_logs()
            # 1%的概率挂起1-5秒钟
            if random.random() < 0.01:
                sleep_time = random.randint(1, 5)
                print(f"Thread {self.id} sleeping for {sleep_time} seconds")
                time.sleep(sleep_time)
                # 加速打印补偿
                self.compensate(sleep_time)
            time.sleep(1)

    def print_logs(self):
        current_time = datetime.now().strftime('%H:%M:%S')
        for _ in range(self.num_prints):
            print(f"{self.id} - {current_time}")

    def compensate(self, sleep_time):
        # 加速打印补偿时间
        compensate_time = sleep_time
        end_time = time.time() + compensate_time
        while time.time() < end_time:
            self.print_logs()
            if random.random() < 0.01:
                compensate_time = random.randint(1, 5)
                print(f"Thread {self.id} sleeping for {compensate_time} seconds during compensation")
                time.sleep(compensate_time)
                end_time += compensate_time

    def stop(self):
        self.stop_event.set()

def main():
    threads = []
    for i in range(1, 6):
        thread = LoggerThread(i, i)
        threads.append(thread)
        thread.start()

    try:
        while True:
            time.sleep(0.1)
    except KeyboardInterrupt:
        for thread in threads:
            thread.stop()

if __name__ == "__main__":
    main()

In [ ]:
import random
import networkx as nx
import matplotlib.pyplot as plt

def create_maze():
    """构造并返回一个有向无环图（DAG），包含20个节点和200条带权重的有向边，权重范围（0，1）"""
    num_nodes = 20
    num_edges = 200
    G = nx.DiGraph()

    # 添加节点
    for i in range(num_nodes):
        G.add_node(i)

    # 添加边，确保图是有向无环图
    while len(G.edges) < num_edges:
        u = random.randint(0, num_nodes - 1)
        v = random.randint(0, num_nodes - 1)
        if u != v and not G.has_edge(u, v) and nx.has_path(G, v, u) == False:  # 防止形成环
            G.add_edge(u, v, weight=random.uniform(0, 1))

    return G

def random_location(G):
    """在图中随机选取一个起始节点并返回"""
    return random.choice(list(G.nodes))

def get_path(G, start_node):
    """输出从起始节点出发的所有可行路径中，权重和最大的路径"""
    def dfs(node, path, weight):
        nonlocal max_path, max_weight
        path.append(node)
        if len(G.out_edges(node)) == 0:
            if weight > max_weight:
                max_weight = weight
                max_path = path[:]
        else:
            for _, neighbor in G.out_edges(node):
                edge_weight = G[node][neighbor]['weight']
                dfs(neighbor, path, weight + edge_weight)
        path.pop()

    max_path = []
    max_weight = 0
    dfs(start_node, [], 0)
    return max_path, max_weight

def start_the_game():
    """封装上述函数作为入口函数"""
    maze = create_maze()

    # 绘制图形
    pos = nx.spring_layout(maze)
    edges = maze.edges(data=True)
    weights = [d['weight'] for (u, v, d) in edges]
    nx.draw(maze, pos, with_labels=True, node_color='lightblue', node_size=500, font_size=10)
    nx.draw_networkx_edge_labels(maze, pos, edge_labels={(u, v): f"{d['weight']:.2f}" for (u, v, d) in edges})
    plt.show()


    start_node = random_location(maze)
    max_path, max_weight = get_path(maze, start_node)
    print(f"Start Node: {start_node}")
    print(f"Max Path: {max_path}")
    print(f"Max Weight: {max_weight}")

    

if __name__ == "__main__":
    start_the_game()